# Goals added prediction

In [1]:
import pandas as pd
import numpy as np



In [8]:
df_raw = pd.read_csv("ASA/goals-added_players_2021-03-03.csv", usecols=range(1,13))
df = df_raw
df

,Player,Team,Season,Position,Minutes,Dribbling,Fouling,Interrupting,Passing,Receiving,Shooting,Goals Added
0,Cristian Pavón,LAG,2020,W,2048,3.34,0.10,-0.10,-0.02,-0.14,0.34,3.52
1,Jordan Morris,SEA,2020,W,1766,1.13,-0.19,-0.21,0.11,2.18,0.15,3.16
2,Diego Rossi,LAFC,2020,W,1851,-0.52,0.03,0.17,0.15,2.01,1.12,2.95
3,Anton Tinnerholm,NYC,2020,FB,2210,0.46,0.15,0.64,1.15,0.20,0.25,2.85
4,Darwin Quintero,HOU,2020,W,1822,0.27,-0.20,-0.21,2.15,-0.23,0.67,2.44
...,...,...,...,...,...,...,...,...,...,...,...,...
604,Mauro Manotas,HOU,2020,ST,1454,-0.58,-0.12,-0.14,-0.88,-0.23,-0.06,-2.01
605,Aljaz Struna,HOU,2020,CB,1575,0.17,-0.47,-1.69,-0.04,-0.26,0.01,-2.28
606,Joe Gyau,CIN,2020,W,1961,0.10,0.08,-0.21,-1.49,-0.53,-0.30,-2.35
607,Yuya Kubo,CIN,2020,W,1412,-0.53,-0.07,-0.14,-0.71,-0.73,-0.20,-2.38
